<a href="https://colab.research.google.com/github/abhijithneilabraham/2000Line/blob/master/automatictextgenerationgpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!git clone https://github.com/openai/gpt-2.git

Cloning into 'gpt-2'...
remote: Enumerating objects: 209, done.
remote: Total 209 (delta 0), reused 0 (delta 0), pack-reused 209
Receiving objects: 100% (209/209), 4.37 MiB | 6.34 MiB/s, done.
Resolving deltas: 100% (109/109), done.


In [9]:
!pip3 install -r requirements.txt

     |████████████████████████████████| 81kB 6.8MB/s 
     |████████████████████████████████| 604kB 20.7MB/s 
     |████████████████████████████████| 51kB 21.5MB/s 
  Created wheel for fire: filename=fire-0.2.1-py2.py3-none-any.whl size=103527 sha256=9185ff49559bd57d659f89772ba20628e45ab7c4b7cae9830020d02488f7e4f4
  Stored in directory: /root/.cache/pip/wheels/31/9c/c0/07b6dc7faf1844bb4688f46b569efe6cafaa2179c95db821da
  Created wheel for regex: filename=regex-2017.4.5-cp36-cp36m-linux_x86_64.whl size=533182 sha256=ac69c11b55b276f32f155f4d25c92513b1e1cc51117d6fed5538e40c2ff9db69
  Stored in directory: /root/.cache/pip/wheels/75/07/38/3c16b529d50cb4e0cd3dbc7b75cece8a09c132692c74450b01
Successfully built fire regex
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [0]:
!python3 download_model.py 345M



In [0]:
!export PYTHONIOENCODING=UTF-8

In [0]:
os.chdir('src')


In [0]:
import json
import os
import numpy as np
import tensorflow as tf
import model, sample, encoder

In [0]:
def interact_model(
    model_name,
    seed,
    nsamples,
    batch_size,
    length,
    temperature,
    top_k,
    models_dir
):
    models_dir = os.path.expanduser(os.path.expandvars(models_dir))
    if batch_size is None:
        batch_size = 1
    assert nsamples % batch_size == 0

    enc = encoder.get_encoder(model_name, models_dir)
    hparams = model.default_hparams()
    with open(os.path.join(models_dir, model_name, 'hparams.json')) as f:
        hparams.override_from_dict(json.load(f))

    if length is None:
        length = hparams.n_ctx // 2
    elif length > hparams.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

    with tf.Session(graph=tf.Graph()) as sess:
        context = tf.placeholder(tf.int32, [batch_size, None])
        np.random.seed(seed)
        tf.set_random_seed(seed)
        output = sample.sample_sequence(
            hparams=hparams, length=length,
            context=context,
            batch_size=batch_size,
            temperature=temperature, top_k=top_k
        )

        saver = tf.train.Saver()
        ckpt = tf.train.latest_checkpoint(os.path.join(models_dir, model_name))
        saver.restore(sess, ckpt)

        while True:
            raw_text = input("Model prompt >>> ")
            while not raw_text:
                print('Prompt should not be empty!')
                raw_text = input("Model prompt >>> ")
            context_tokens = enc.encode(raw_text)
            generated = 0
            for _ in range(nsamples // batch_size):
                out = sess.run(output, feed_dict={
                    context: [context_tokens for _ in range(batch_size)]
                })[:, len(context_tokens):]
                for i in range(batch_size):
                    generated += 1
                    text = enc.decode(out[i])
                    print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
                    print(text)
            print("=" * 80)

In [0]:
interact_model(
    '345M',
    None,
    1,
    1,
    300,
    1,
    0,
    '/content/gpt-2/models'
)

W0831 13:51:26.166046 140216978483072 deprecation_wrapper.py:119] From /content/gpt-2/src/sample.py:51: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.

W0831 13:51:26.167520 140216978483072 deprecation_wrapper.py:119] From /content/gpt-2/src/model.py:148: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0831 13:51:26.178414 140216978483072 deprecation_wrapper.py:119] From /content/gpt-2/src/model.py:152: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

W0831 13:51:26.229646 140216978483072 deprecation_wrapper.py:119] From /content/gpt-2/src/model.py:36: The name tf.rsqrt is deprecated. Please use tf.math.rsqrt instead.

W0831 13:51:31.861155 140216978483072 deprecation.py:323] From /content/gpt-2/src/sample.py:64: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
W0831 13

Model prompt >>> Me and my friend are chilling together in Miami
======================================== SAMPLE 1 ========================================
 but couldn't really get a good feed to share in terms of talent and connection. We wanted to help our friend earn money through his dorm rooms right? So I called Babo35 and got him to sign on the basis of his nude & mobile photos. That gets a good run: Babo35 submitted his naked pics to us and (yourdubstepgoer) that was awesome♥ Thanks AWOOOOO!!

Then a few months later Babo35 asked me if he could start booking dates the following year ("I've been traveling for the past seven years) for a bit to smooth things over."

This simulates a little bit of my younger self's 'boyfriend plan" in which he traveled across multiple countries with girlfriends by secretly scouting out what's available across the globe to recruit guys and brainstorm dates. Before I leave my house this morning to sell drugs or pot, Babo35 will be with me for 35 days